In [25]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [26]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

In [27]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [28]:
import os
PERSIST_DIR = "lyrics_store2"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("47923_BTS.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [29]:
query_engine = index.as_query_engine()

In [30]:
qa_template_str = """
    You are an expert in K-Pop music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [31]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [32]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

description = """
A set of lyrics for songs from the Korean Group BTS. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""
tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="BTS_songs_lyrics",
            description=description,
            return_direct=False
        )
    )
]
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [33]:
response = agent.chat("¿Qué canciones de BTS hablan de nostalgia?")

Added user message to memory: ¿Qué canciones de BTS hablan de nostalgia?
=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"nostalgia"}
Got output: - La canción "Bad Decisions" de BTS no tiene un tema específico de nostalgia, pero en su letra se puede apreciar un tono de melancolía y anhelo por el amor y la compañía de la persona amada.
- En la canción se mencionan frases como "And it feels like I can't seem to ever get you out of my mind" y "I want you, ooh, baby, all of the time", que reflejan un sentimiento de nostalgia y deseo constante por la persona amada.
- A pesar de que la canción tiene un ritmo animado y pegajoso, las letras transmiten una sensación de añoranza y nostalgia por la presencia y el amor de esa persona especial.

=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"nostalgia"}
Got output: - La canción "Bad Decisions" de BTS no tiene un tono de nostalgia, más bien tiene un tono de deseo y pasión.
- En la can

In [34]:
response.response

'- La canción "Bad Decisions" de BTS no tiene un tono de nostalgia, más bien tiene un tono sensual y provocativo.\n- En la canción, se mencionan frases como "I want you, ooh, baby, all of the time" y "Let\'s make some bad decisions", lo que refleja un deseo intenso y pasional.\n- La letra de la canción se centra en la atracción física y la intensidad del deseo entre dos personas, sin mencionar sentimientos de nostalgia.\n\n- La canción "Bad Decisions" de BTS no tiene un tono de nostalgia, más bien tiene un tono de deseo y pasión.\n- En la canción, se mencionan frases como "I want your love, I want your name" y "I want you, ooh, baby, all of the time", lo que refleja un deseo intenso por la persona amada.\n- El verso de Snoop Dogg también refleja esta pasión y deseo, con frases como "Let\'s do things you always wanted" y "I want you, ooh, baby, all of the time".\n- La canción tiene un tono sensual y provocativo, con referencias a hacer "malas decisiones" y a querer estar con la persona 

In [35]:
response = agent.chat("¿Qué canciones hablan sobre el amor propio?")

Added user message to memory: ¿Qué canciones hablan sobre el amor propio?
=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"self-love"}
Got output: - La canción "Outro: Luv In Skool" de BTS aborda el tema del amor propio y la autoaceptación.
- En la letra se mencionan frases como "I just wanna love ya" y "Me amo a mí mismo", enfatizando la importancia de amarse a uno mismo.
- Se hace referencia a la idea de conocerse a uno mismo y aceptarse tal como se es.
- El tono de la canción es positivo y empoderador, animando a los oyentes a amarse a sí mismos y a no depender de la validación externa.

=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"self-love"}
Got output: - La canción "Outro: Luv In Skool" de BTS aborda el tema del amor propio y la importancia de amarse a uno mismo.
- En la letra se mencionan frases como "I just wanna love ya" y "Me amo, me amo, me amo a mí mismo", resaltando la idea de valorarse y aceptarse tal com

### TAREA 15-10-2024

In [1]:
import wikipedia
from llama_index.core import Document, VectorStoreIndex
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def get_artist_documents(filename: str) -> list[Document]:
  with open(filename) as file:
    data = file.read()
  songs = data.split("===")
  artist = songs.pop(0).strip()
  
  documents = [
    Document(
      text=song,
      metadata={
        "category":"music",
        "artist": artist,
      }
    )
    for song in songs
  ]    
  return documents

In [3]:
bts_songs_file = '47923_BTS.txt'

lyrics_gotten = get_artist_documents(bts_songs_file)

In [4]:
wikipedia.set_lang('es')

bts_content = wikipedia.page('BTS').content

with open('BTS_info.txt', 'w', encoding='utf-8') as file:
  file.write("== Nombre ==\n")
  file.write(bts_content)

In [5]:
def get_bts_documents(filename: str) -> list[Document]:
  with open(filename, 'r', encoding='utf-8') as file:
    data = file.read()
  
  sections = data.split("===")
  sections = [section.strip() for section in sections if section.strip()]
  
  artist_line = sections.pop(0)
  artist = artist_line.split("==")[1].strip() if "==" in artist_line else "BTS"
  
  documents = [
    Document(
      text=section.strip(),
      metadata={
        "category": "music",
        "artist": artist
      }
    )
    for section in sections
  ]
  
  return documents

In [6]:
bts_info_documents = get_bts_documents('BTS_info.txt')

index = VectorStoreIndex.from_documents(bts_info_documents)

query_engine = index.as_query_engine()

In [7]:
description = """
This tool provides information about BTS, including song lyrics and biographical data obtained from Wikipedia.
It accepts natural language questions as input.
MANDATORY: Provide the response to the user exactly as it is, preserving the format; do not attempt to summarize when using this tool.
"""

In [8]:
tools = [
  QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
      name="BTS_Info_and_Lyrics",
      description=description,
      return_direct=False
    )
  )
]

bts_agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [9]:
response = bts_agent.chat('¿Puedes describir los temas líricos de BTS?')
print(response)

response = bts_agent.chat('¿Cual es el origen de BTS?')
print(response)

Added user message to memory: ¿Puedes describir los temas líricos de BTS?
=== Calling Function ===
Calling function: BTS_Info_and_Lyrics with args: {"input":"Describe los temas líricos de BTS"}
Got output: BTS aborda una amplia gama de temas en sus letras, incluyendo la salud mental, el deseo de aceptación en la sociedad, la soledad, la depresión, la presión de ser joven y exitoso, la apatía social, la crítica al materialismo, la obsesión de los fanáticos, la disparidad generacional, y la lucha de las minorías. También han tratado temas políticos como la dificultad de los millenials para encontrar empleo y vivienda, así como la crítica a las instituciones y a las generaciones mayores.

BTS aborda una amplia gama de temas en sus letras, incluyendo la salud mental, el deseo de aceptación en la sociedad, la soledad, la depresión, la presión de ser joven y exitoso, la apatía social, la crítica al materialismo, la obsesión de los fanáticos, la disparidad generacional, y la lucha de las mino